In [ ]:
from dotenv import load_dotenv
import openai
from openai import OpenAI
import json
import os
import requests
from pypdf import PdfReader
import gradio as gr
from IPython.display import Markdown, display

In [ ]:
'''pushover example
This example shows how to send a message with an attachment using the Pushover API.
import requests
r = requests.post("https://api.pushover.net/1/messages.json", data = {
  "token": "APP_TOKEN",
  "user": "USER_KEY",
  "message": "hello world"
},
files = {
  "attachment": ("image.jpg", open("your_image.jpg", "rb"), "image/jpeg")
})
print(r.text)
'''

In [ ]:
load_dotenv(override=True)
#openai = OpenAI()

In [ ]:
# Load the API keys from environment variables
groq_api_key = os.getenv('GROQ_API_KEY')
if groq_api_key:
    print(f"Groq API Key exists and begins {groq_api_key[:4]}")
else:
    print("Groq API Key not set (and this is optional)")

In [ ]:
# For pushover
pushover_user = os.getenv("PUSHOVER_USER")
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_url = "https://api.pushover.net/1/messages.json"

In [ ]:
def push(message):
    print(f"Push: {message}")
    payload = {"user": pushover_user, "token": pushover_token, "message": message}
    requests.post(pushover_url, data=payload)


In [ ]:
request = "Please come up with a challenging, nuanced question that I can ask a number of LLMs to evaluate their intelligence. "
request += "Answer only with the question, no explanation."
messages = [{"role": "user", "content": request}]

In [ ]:
groq = OpenAI(api_key=groq_api_key, base_url="https://api.groq.com/openai/v1")
model_name = "llama-3.1-8b-instant"
#model_name ="meta-llama/llama-4-scout-17b-16e-instruct"
response = groq.chat.completions.create(model=model_name, messages=messages)
answer = response.choices[0].message.content

display(Markdown(answer))

In [ ]:
def record_user_details(email, name="Name not provided", notes="not provided"):
    push(f"Recording interest from {name} with email {email} and notes {notes}")
    return {"recorded": "ok"}

In [ ]:
def record_unknown_question(question):
    push(f"Recording {question} asked that I couldn't answer")
    return {"recorded": "ok"}

In [ ]:
record_user_details_json = {
    "name": "record_user_details",
    "description": "Use this tool to record that a user is interested in being in touch and provided an email address",
    "parameters": {
        "type": "object",
        "properties": {
            "email": {
                "type": "string",
                "description": "The email address of this user"
            },
            "name": {
                "type": "string",
                "description": "The user's name, if they provided it"
            }
            ,
            "notes": {
                "type": "string",
                "description": "Any additional information about the conversation that's worth recording to give context"
            }
        },
        "required": ["email"],
        "additionalProperties": False
    }
}



In [ ]:
record_unknown_question_json = {
        "type": "function",
        "function": {
            "name": "record_unknown_question",
            "description": "Precisely record questions that cannot be directly answered",
            "parameters": {
                "type": "object",
                "properties": {
                    "question": {
                        "type": "string",
                        "description": "Exact text of the question"
                    }
                },
                "required": ["question"],
                "additionalProperties": False
            }
        }
    }

In [ ]:
tools = [{"type": "function", "function": record_user_details_json},
        {"type": "function", "function": record_unknown_question_json}]

In [ ]:
def handle_tool_calls(tool_calls):
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        print(f"Tool called: {tool_name}", flush=True)
        print(f"Arguments: {arguments}", flush=True)
        tool = globals().get(tool_name)
        result = tool(**arguments) if tool else {}
        results.append({"role": "tool","content": json.dumps(result),"tool_call_id": tool_call.id})
    return results

In [ ]:
name = os.getenv("MY_NAME")
name

In [ ]:
with open("data/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

with open("data/resume.md", "r", encoding="utf-8") as f:
    resume = f.read()

print(f"Summary: {summary[:100]}...")  # Print first 100 characters for brevity
print(f"Resume: {resume[:100]}...")  # Print first 100 characters for brevity

In [ ]:
system_prompt = f"""You are {name}. You are answering questions about yourself on your professional website.

## STRICT RULES:
1. NEVER invent, assume, or extrapolate information not explicitly written in your summary or resume
2. ONLY answer questions with facts that can be directly quoted or paraphrased from your provided materials
3. If asked about strengths/weaknesses, future plans, soft skills, or another kind of questions NOT explicitly mentioned in your materials, USE the record_unknown_question tool to save the question and politely decline
4. For ANY question about information not directly stated, respond: "I don't have that specific information to share right now. Feel free to reach out to me directly via email if you'd like to discuss this further."

## CONTACT COLLECTION (BE SUBTLE):
- Only suggest contact exchange when users explicitly mention opportunities, collaborations, or wanting to work together
- Don't push for contact info after every interaction
- When appropriate, say something like: "That sounds interesting! If you'd like to discuss this further, I'd be happy to connect. What's your name and email?"
- ONLY use record_user_details when you have BOTH name and email
- Once you have contact details, don't ask for them again in the same conversation.

## SPEAK AS YOURSELF:
- Use "I" statements: "I worked at...", "My experience includes...", "I studied..."
- Be conversational but professional
- Don't refer to yourself in third person

## ACCEPTABLE RESPONSES:
- Direct facts from your resume using first person
- Information explicitly stated in your summary
- Redirecting to email contact for detailed discussions

## TOOLS USAGE:
- Use record_unknown_question for ANY question you cannot answer from your materials
- Use record_user_details ONLY when you have both name and email AND the user has expressed genuine business interest
- Don't mention these tools to users

## Your Summary:
{summary}

## Your Resume:
{resume}

Answer as yourself, stay within the facts provided, and only collect contact information when there's a clear business opportunity being discussed."""

In [ ]:
def chat(message, history):
    # Remove metadata field from messages
    # it is added by gradio but it is not compatible with grog's api
    cleaned_history = [
        {k: v for k, v in msg.items() if k in ['role', 'content']} 
        for msg in history
    ]
    messages = [{"role": "system", "content": system_prompt}] + cleaned_history + [{"role": "user", "content": message}]

    done = False
    while not done:
        print (messages[1:])  # Print the messages excluding the system prompt for clarity
        # This is the call to the LLM - see that we pass in the tools json

        try:
            response = groq.chat.completions.create(model=model_name, messages=messages, tools=tools, temperature=0.0)

            finish_reason = response.choices[0].finish_reason
            
            # If the LLM wants to call a tool, we do that!
            
            if finish_reason=="tool_calls":
                print("Tool calls detected, handling them...", flush=True)
                message = response.choices[0].message
                tool_calls = message.tool_calls
                print(f"Tool calls: {response.choices[0]}", flush=True)
                results = handle_tool_calls(tool_calls)
                messages.append(message)
                messages.extend(results)
            else:
                done = True
        except openai.error.OpenAIError as e:
            print(f"An error occurred: {e}")
    return response.choices[0].message.content

In [ ]:
gr.ChatInterface(chat, type="messages").launch()